# Building networks

For the industry benchmark systems we provide the ligands with different sets of partial charges along with LOMAP networks generated with Kartograf atom mappings, this notebook then combines the two to make an `AlchemicalNetwork` ready for simulation.

Note this assumes you are using a protocol which runs a bond and solvent leg as two different transformations.

In [9]:
import pathlib
import gufe
from openfe.protocols.openmm_rfe import RelativeHybridTopologyProtocol
from rdkit import Chem
import openfe

In [ ]:
# declare the charge settings and the system you want to generate the network for
CHARGE_MODEL = "antechamber_am1bcc" 
# or
#CHARGE_MODEL = "openeye_elf10"
SYSTEM_GROUP = "jacs_set"
SYSTEM_NAME = "tyk2"

In [8]:
# define the protocol to use
settings = RelativeHybridTopologyProtocol.default_settings()
settings.protocol_repeats = 1
protocol = RelativeHybridTopologyProtocol(settings=settings)

In [ ]:
# find the system
dataset_dir = pathlib.Path("../openfe_benchmarks/data/industry_benchmark_systems")
system_dir = dataset_dir.joinpath(SYSTEM_GROUP, SYSTEM_NAME)

# load the network 
network = gufe.LigandNetwork.from_graphml(system_dir.joinpath("lomap_network.graphml").read_text())

# load the ligands with charges
supplier = Chem.SDMolSupplier(system_dir.joinpath(f"ligands_{CHARGE_MODEL}.sdf"), removeHs=False)
ligands_by_name = {}
for mol in supplier:
    ofe_mol = gufe.SmallMoleculeComponent.from_rdkit(mol)
    ligands_by_name[ofe_mol.name] = ofe_mol

# load the protein
protein = gufe.ProteinComponent.from_pdb_file(system_dir.joinpath("protein.pdb").as_posix())

# check for cofactors
co_file = system_dir.joinpath(f"cofactors_{CHARGE_MODEL}.sdf")
cofactors = None
if co_file.exists():
    cofactors = []
    supplier = Chem.SDMolSupplier(co_file, removeHs=False)
    for mol in supplier:
        cofactors.append(gufe.SmallMoleculeComponent.from_rdkit(mol))

# define the standard solvent
solvent = gufe.SolventComponent()

# build the transforms
transformations = []
for edge in network.edges:
    # make the edge again using ligands with charges
    new_edge = gufe.LigandAtomMapping(
        componentA=ligands_by_name[edge.componentA.name],
        componentB=ligands_by_name[edge.componentB.name],
        componentA_to_componentB=edge.componentA_to_componentB
    )

    # create the transformations for the bound and solvent legs
    for leg in ["solvent", "complex"]:
        system_a_dict = {
            "ligand": new_edge.componentA, "solvent": solvent
        }
        system_b_dict = {
            "ligand": new_edge.componentB, "solvent": solvent
        }
        if leg == "complex":
            system_a_dict["protein"] = protein
            system_b_dict["protien"] = protein

            if cofactors is not None:
                for i, cofactor in enumerate(cofactors):
                    cofactor_name = f"cofactor_{i}"
                    system_a_dict[cofactor_name] = cofactor
                    system_b_dict[cofactor_name] = cofactor

        system_a = gufe.ChemicalSystem(system_a_dict)
        system_b = gufe.ChemicalSystem(system_b_dict)

        name = f"{leg}_{new_edge.componentA.name}_{new_edge.componentB.name}"
        transformation = openfe.Transformation(
            stateA=system_a,
            stateB=system_b,
            mapping=new_edge,
            protocol=protocol,
            name=name
        )
        transformations.append(transformation)

# create the network, profit! 
alchemical_network = gufe.AlchemicalNetwork(edges=transformations)